# SnapTo3D Spatial‑Planning Pipeline

Flow: 
```
USER prompt -> LLM (scene‑graph JSON) -> Constraint Solver -> Blender CodeGen -> Render -> VLM Verifier
```

## 0  Setup and dependencies

In [55]:
# ↳ Run once per environment
!source spatial-env/bin/activate
# !pip install -r requirements.txt

zsh:source:1: no such file or directory: spatial-env/bin/activate


## 1  Scene‑graph schema & helper classes

In [56]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv

from ortools.sat.python import cp_model
import time

import subprocess
import pathlib
import cloudinary
import cloudinary.uploader

from cloudinary.utils import cloudinary_url
import os
from loguru import logger

load_dotenv()

True

## 2  LLM reasoning → JSON

In [57]:
MODEL = 'gpt-4'

OpenAPIClient = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_to_scene(prompt: str) -> dict:
    function = {
    "name": "create_scene_graph",
    "description": "Creates a comprehensive 3D scene graph with room architecture, furniture, and materials from a description",
    "parameters": {
        "type": "object",
        "properties": {
            "metadata": {
                "type": "object",
                "description": "Scene-level metadata",
                "properties": {
                    "projectId": {"type": "string", "description": "Unique scene/room ID"},
                    "roomType": {"type": "string", "enum": ["bedroom", "lounge", "bathroom", "kitchen", "office", "dining"], "description": "Type of room"},
                    "units": {"type": "string", "description": "Linear unit symbol (e.g., 'm', 'ft')"},
                    "axes": {
                        "type": "object",
                        "properties": {
                            "right": {"type": "string", "description": "Axis pointing right (e.g., 'X+')"},
                            "forward": {"type": "string", "description": "Axis pointing forward (e.g., 'Y+')"},
                            "up": {"type": "string", "description": "Axis pointing up (e.g., 'Z+')"}
                        },
                        "required": ["right", "forward", "up"]
                    }
                },
                "required": ["projectId", "roomType", "units", "axes"]
            },
            "room": {
                "type": "object",
                "description": "Architectural shell definition",
                "properties": {
                    "dims": {
                        "type": "object",
                        "properties": {
                            "w": {"type": "number", "description": "Interior width"},
                            "d": {"type": "number", "description": "Interior depth"},
                            "h": {"type": "number", "description": "Interior height"}
                        },
                        "required": ["w", "d", "h"]
                    },
                    "walls": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "id": {"type": "string", "description": "Stable wall identifier"},
                                "start": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "World-space XYZ of wall start"},
                                "end": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "World-space XYZ of wall end"},
                                "thickness": {"type": "number", "description": "Wall thickness"},
                                "mat": {"type": "string", "description": "Material ID"},
                                "mesh": {"type": ["string", "null"], "description": "Optional custom mesh path"}
                            },
                            "required": ["id", "start", "end", "thickness", "mat"]
                        }
                    },
                    "openings": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "id": {"type": "string", "description": "Door/window identifier"},
                                "wallId": {"type": "string", "description": "Host wall's ID"},
                                "offset": {"type": "number", "description": "Distance from wall start"},
                                "w": {"type": "number", "description": "Opening width"},
                                "h": {"type": "number", "description": "Opening height"},
                                "sill": {"type": "number", "description": "Z-offset of bottom edge"}
                            },
                            "required": ["id", "wallId", "offset", "w", "h", "sill"]
                        }
                    },
                    "floorMat": {"type": "string", "description": "Default floor material ID"}
                },
                "required": ["dims", "walls", "floorMat"]
            },
            "furniture": {
                "type": "array",
                "description": "Furnishings and decor objects",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Instance name"},
                        "catalogId": {"type": "string", "description": "SKU/external reference"},
                        "category": {"type": "string", "description": "Furniture category (e.g., bed, sofa, sink)"},
                        "dims": {
                            "type": ["object", "null"],
                            "properties": {
                                "w": {"type": "number"},
                                "d": {"type": "number"},
                                "h": {"type": "number"},
                                "source": {"type": "string", "enum": ["catalog", "inferred", "user"]}
                            }
                        },
                        "xform": {
                            "type": "object",
                            "description": "Blender-ready transform",
                            "properties": {
                                "pos": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Translation XYZ"},
                                "rot": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Euler rotation degrees XYZ"},
                                "scale": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Non-uniform scale factors"}
                            },
                            "required": ["pos", "rot", "scale"]
                        },
                        "mesh": {"type": ["string", "null"], "description": "File path to mesh/prefab"},
                        "matOverride": {"type": ["string", "null"], "description": "Optional material swap"},
                        "tags": {"type": "array", "items": {"type": "string"}, "description": "Semantic labels"}
                    },
                    "required": ["id", "catalogId", "category", "xform"]
                }
            },
            "constraints": {
                "type": "array",
                "description": "Declarative layout rules",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Human-readable slug"},
                        "type": {"type": "string", "enum": ["offset", "align", "inside", "groupAlign"], "description": "Constraint type"},
                        "targets": {"type": "array", "items": {"type": "string"}, "description": "IDs of objects/walls involved"},
                        "axis": {"type": "string", "enum": ["X", "Y", "Z"], "description": "Axis for offset/align rules"},
                        "distance": {"type": "number", "description": "Gap size"},
                        "relativeTo": {"type": "string", "description": "Anchoring object ID"},
                        "alignAxis": {"type": "string", "description": "Axis to match in groupAlign"}
                    },
                    "required": ["id", "type", "targets"]
                }
            },
            "materials": {
                "type": "array",
                "description": "Shared material palette",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Material name"},
                        "color": {"type": "array", "items": {"type": "number", "minimum": 0, "maximum": 1}, "minItems": 3, "maxItems": 3, "description": "RGB 0-1 values"},
                        "tex": {"type": ["string", "null"], "description": "Path to texture map"},
                        "roughness": {"type": ["number", "null"], "minimum": 0, "maximum": 1, "description": "PBR roughness 0-1"},
                        "scaleUV": {"type": ["number", "null"], "description": "Texture tiling factor"}
                    },
                    "required": ["id"]
                }
            }
        },
        "required": ["metadata", "room", "furniture", "materials"]
    }
}
    
    system_message = """
                        You are a 3D spatial planning assistant. 
                        Generate realistic room layouts with proper furniture placement, considering spatial relationships, accessibility, and design principles.
                        Always include complete scene graphs with dimensions, transforms, and materials.
        """
    
    try:
        response = OpenAPIClient.chat.completions.create(
            model=MODEL,
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
            functions=[function],
            function_call={'name':'create_scene_graph'}
        )
        
        if not response.choices or not response.choices[0].message.function_call:
            raise ValueError("No valid response from API")
            
        logger.info(f"Scene graph generated successfully ...")
        return json.loads(response.choices[0].message.function_call.arguments)
    except Exception as e:
        logger.error(f"Error in llm_to_scene: {str(e)}")
        raise

## 3  Constraint solver → absolute coordinates

In [58]:
GRID = 100 
MAX_RETRIES = 5

def solve_scene(scene: dict) -> dict:
    """
    Solve scene constraints. Returns placements dict on success, False on error.
    """
    logger.info(f"Solving scene constraints ...")

    try:
        m = cp_model.CpModel()
        pos, half = {}, {}

        if scene.get('constraints') is None:
            logger.error(f"No constraints found in scene")
            return False

        # Extract furniture items and convert to expected format
        for item in scene['furniture']:
            # Handle different dimension formats
            if 'w' in item['dims'] and 'l' in item['dims']:
                w = item['dims']['w']
                d = item['dims']['l']  # 'l' (length) maps to 'd' (depth)
            elif 'balloon' in item['dims']:  # Special case for lamp
                w = item['dims']['balloon']['r'] * 2
                d = item['dims']['balloon']['r'] * 2
            else:
                w, d = 0.5, 0.5 
                
            half[item['id']] = (int(w*GRID/2), int(d*GRID/2))
            pos[item['id']]  = (                     
                m.NewIntVar(0, int(scene['room']['dims']['w']*GRID), f'x_{item["id"]}'),
                m.NewIntVar(0, int(scene['room']['dims']['d']*GRID), f'y_{item["id"]}')
            )

        wall_positions = {}
        for wall in scene['room']['walls']:
            if wall['id'] == 'wall1':  # Bottom wall (Y=0)
                wall_positions[wall['id']] = {'y': 0}
            elif wall['id'] == 'wall2':  # Left wall (X=0)
                wall_positions[wall['id']] = {'x': 0}
            elif wall['id'] == 'wall3':  # Right wall
                wall_positions[wall['id']] = {'x': scene['room']['dims']['w']}
            elif wall['id'] == 'wall4':  # Top wall
                wall_positions[wall['id']] = {'y': scene['room']['dims']['d']}

        # relational constraints --------------------------------------------------
        for c in scene['constraints']:
            if c['type'] == 'offset':
                for target in c['targets']:
                    if target not in pos:
                        logger.error(f"Target '{target}' not found in furniture")
                        return False
                        
                    distance = int(c['distance'] * GRID)
                    x, y = pos[target]
                    w, d = half[target]
                    
                    if c['relativeTo'] not in wall_positions:
                        print(f"Error: Wall '{c['relativeTo']}' not found")
                        return False
                        
                    wall_pos = wall_positions[c['relativeTo']]
                    
                    if c['axis'] == 'X' and 'x' in wall_pos:
                        if wall_pos['x'] == 0:  # Left wall
                            m.Add(x - w == distance)
                        else:  # Right wall
                            m.Add(x + w == int(wall_pos['x'] * GRID) - distance)
                    elif c['axis'] == 'Y' and 'y' in wall_pos:
                        if wall_pos['y'] == 0:  # Bottom wall
                            m.Add(y - d == distance)
                        else:  # Top wall
                            m.Add(y + d == int(wall_pos['y'] * GRID) - distance)
                            
            elif c['type'] == 'align':
                # Handle alignment constraints
                for target in c['targets']:
                    if target not in pos:
                        logger.error(f"Target '{target}' not found in furniture")
                        return False
                    if c['relativeTo'] not in pos:
                        logger.error(f"RelativeTo '{c['relativeTo']}' not found in furniture")
                        return False
                        
                    xa, ya = pos[target]
                    xb, yb = pos[c['relativeTo']]
                    
                    if c.get('alignAxis') == 'Y':
                        # Align along Y axis (same Y coordinate)
                        m.Add(ya == yb)
                    elif c.get('alignAxis') == 'X':
                        # Align along X axis (same X coordinate)
                        m.Add(xa == xb)

        # NON-OVERLAP (boolean disjunction) ---------------------------------------
        ids = list(pos.keys())
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                A, B = ids[i], ids[j]
                xa, ya = pos[A]; xb, yb = pos[B]
                wa, da = half[A]; wb, db = half[B]

                if (A == 'lamp1' and B == 'desk1') or (A == 'desk1' and B == 'lamp1'):
                    continue

                left   = m.NewBoolVar(f'{A}_left_{B}')
                right  = m.NewBoolVar(f'{A}_right_{B}')
                front  = m.NewBoolVar(f'{A}_front_{B}')
                behind = m.NewBoolVar(f'{A}_behind_{B}')

                m.Add(xa + wa <= xb - wb).OnlyEnforceIf(left)
                m.Add(xb + wb <= xa - wa).OnlyEnforceIf(right)
                m.Add(ya + da <= yb - db).OnlyEnforceIf(front)
                m.Add(yb + db <= ya - da).OnlyEnforceIf(behind)

                m.AddBoolOr([left, right, front, behind])

        # -------------------------------------------------------------------------
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 10.0  # Add timeout
        status = solver.Solve(m)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            return {oid: (solver.Value(x)/GRID, solver.Value(y)/GRID)
                    for oid,(x,y) in pos.items()}
        else:
            logger.error(f"Solver failed with status: {solver.StatusName(status)}")
            if status == cp_model.INFEASIBLE:
                logger.error("The constraints are contradictory and cannot be satisfied.")
            return False
            
    except Exception as e:
        logger.error(f"Error in solve_scene: {str(e)}")
        return False

## 4  CodeGen → Blender .py script

In [59]:
def gen_blender_py(scene: dict, placements: dict, outfile='assets/blender/script.py'):
    """
    Generate a standalone Blender Python script from a scene graph plus placements.

    Args:
        scene:       Dictionary with room / walls / furniture / materials
        placements:  Dict mapping furniture IDs → (x, y) positions (metres)
        outfile:     Filepath for the generated .py script
    """
    lines = [
        "import bpy",
        "from mathutils import Vector",
        "import math",
        "",
        "# -------------------------------------------------- Reset scene",
        "bpy.ops.object.select_all(action='SELECT')",
        "bpy.ops.object.delete(use_global=False)",
        "",
    ]

    logger.info("Generating Blender script → %s", outfile)

    # ---- room envelope -----------------------------------------------------
    room_dims  = scene.get('room', {}).get('dims', {'w': 4, 'd': 2, 'h': 3})
    room_w     = room_dims['w']
    room_d     = room_dims['d']
    room_h     = room_dims['h']

    # ---- CAMERA (auto-framing isometric) -----------------------------------
    diag       = max(room_w, room_d)
    cam_loc    = (room_w / 2 + diag * 1.20,
                  -diag * 1.20,
                  room_h + diag * 0.90)
    cam_target = (room_w / 2, room_d / 2, room_h / 2)

    lines += [
        "# -------------------------------------------------- Camera",
        "cam_data  = bpy.data.cameras.new(name='Camera')",
        "cam_data.lens = 35",
        "cam_obj   = bpy.data.objects.new('Camera', cam_data)",
        "bpy.context.collection.objects.link(cam_obj)",
        f"cam_obj.location = {cam_loc}",
        "",
        "# Aim camera at centre of room (65° elevation, 45° azimuth iso-view)",
        f"direction = Vector({cam_target}) - cam_obj.location",
        "cam_obj.rotation_euler = direction.to_track_quat('-Z', 'Y').to_euler()",
        "bpy.context.scene.camera = cam_obj",
        "",
    ]

    # ---- LIGHT -------------------------------------------------------------
    lines += [
        "# -------------------------------------------------- Lighting",
        "bpy.ops.object.light_add(type='SUN', location=(5, -5, 10))",
        "bpy.context.active_object.data.energy = 2.0",
        "",
    ]

    # ---- RENDER SETTINGS ---------------------------------------------------
    lines += [
        "# -------------------------------------------------- Render settings",
        "scene = bpy.context.scene",
        "scene.render.engine = 'CYCLES'",
        "scene.cycles.samples = 64",
        "scene.render.resolution_x = 1920",
        "scene.render.resolution_y = 1080",
        "",
    ]

    # ---- FLOOR -------------------------------------------------------------
    lines += [
        "# -------------------------------------------------- Floor",
        f"bpy.ops.mesh.primitive_cube_add(location=({room_w/2}, {room_d/2}, -0.05))",
        "floor = bpy.context.active_object",
        "floor.name = 'Floor'",
        f"floor.scale = ({room_w/2}, {room_d/2}, 0.05)",
        "",
    ]

    # ---- WALLS (optional) ---------------------------------------------------
    if 'walls' in scene.get('room', {}):
        lines.append("# -------------------------------------------------- Walls")
        for i, wall in enumerate(scene['room']['walls']):
            wall_id   = wall.get('id', f'wall_{i+1}')
            start, end = wall['start'], wall['end']
            thick      = wall.get('thickness', 0.2)

            if start[0] != end[0]:        # horizontal wall (X axis length)
                cx = (start[0] + end[0]) / 2
                cy = start[1]
                cz = room_h / 2
                sx = abs(end[0] - start[0]) / 2
                sy = thick / 2
                sz = room_h / 2
            else:                         # vertical wall (Y axis length)
                cx = start[0]
                cy = (start[1] + end[1]) / 2
                cz = room_h / 2
                sx = thick / 2
                sy = abs(end[1] - start[1]) / 2
                sz = room_h / 2

            lines += [
                f"bpy.ops.mesh.primitive_cube_add(location=({cx}, {cy}, {cz}))",
                "wall = bpy.context.active_object",
                f"wall.name = '{wall_id}'",
                f"wall.scale = ({sx}, {sy}, {sz})",
                "",
            ]

    # ---- FURNITURE ---------------------------------------------------------
    lines.append("# -------------------------------------------------- Furniture")
    for item in scene.get('furniture', []):
        fid  = item['id']
        x, y = placements.get(fid, (item.get('xform', {}).get('pos', [1, 0, 1])[:2]))
        dims = item.get('dims', {})
        w, d, h = dims.get('w', 1.0), dims.get('d', dims.get('l', 1.0)), dims.get('h', 0.5)
        z   = h / 2
        rot = item.get('xform', {}).get('rot', [0, 0, 0])
        cat = item.get('category', '').lower()

        if 'lamp' in cat:
            primitive = "primitive_cylinder_add"
            scale     = f"({w/2:.2f}, {d/2:.2f}, {h/2:.2f})"
        else:
            primitive = "primitive_cube_add"
            scale     = f"({w/2:.2f}, {d/2:.2f}, {h/2:.2f})"

        lines += [
            f"# {fid}  ({cat})",
            f"bpy.ops.mesh.{primitive}(location=({x:.2f}, {y:.2f}, {z:.2f}))",
            "obj = bpy.context.active_object",
            f"obj.name = '{fid}'",
            f"obj.scale = {scale}",
            f"obj.rotation_euler = ({rot[0]}, {rot[1]}, {rot[2]})",
            "",
        ]

    # ---- MATERIALS ---------------------------------------------------------
    if 'materials' in scene:
        lines.append("# -------------------------------------------------- Materials")
        for mat in scene['materials']:
            mid, col = mat['id'], mat.get('color', [200, 200, 200])
            rough    = mat.get('roughness', 0.5)
            r, g, b  = [c/255 for c in col]
            lines += [
                f"mat = bpy.data.materials.new(name='{mid}')",
                "mat.use_nodes = True",
                "bsdf = mat.node_tree.nodes['Principled BSDF']",
                f"bsdf.inputs['Base Color'].default_value = ({r:.3f}, {g:.3f}, {b:.3f}, 1.0)",
                f"bsdf.inputs['Roughness'].default_value = {rough}",
                "",
            ]

        # auto-assign floor / wall mats if specified
        if 'floorMat' in scene.get('room', {}):
            fm = scene['room']['floorMat']
            lines += [
                "floor = bpy.data.objects.get('Floor')",
                f"if floor and '{fm}' in bpy.data.materials:",
                f"    floor.data.materials.append(bpy.data.materials['{fm}'])",
                "",
            ]
        for wall in scene.get('room', {}).get('walls', []):
            if 'mat' in wall:
                mid = wall['mat']
                wid = wall['id']
                lines += [
                    f"w = bpy.data.objects.get('{wid}')",
                    f"if w and '{mid}' in bpy.data.materials:",
                    f"    w.data.materials.append(bpy.data.materials['{mid}'])",
                    "",
                ]

    # ---- SAVE + RENDER -----------------------------------------------------
    lines += [
        "# -------------------------------------------------- Save & Render",
        "import os",
        "os.makedirs('assets/renders', exist_ok=True)",
        "bpy.ops.wm.save_as_mainfile(filepath='assets/renders/scene.blend')",
        "",
        "bpy.context.scene.render.filepath = 'assets/renders/scene.png'",
        "bpy.ops.render.render(write_still=True)",
    ]

    logger.info("…%d lines of Blender script generated", len(lines))

    with open(outfile, 'w') as f:
        f.write('\n'.join(lines))

    return outfile

## 5  Headless Blender render

In [60]:
cloudinary.config(
    cloud_name=os.getenv('CLOUDINARY_CLOUD_NAME'),
    api_key=os.getenv('CLOUDINARY_API_KEY'),
    api_secret=os.getenv('CLOUDINARY_API_SECRET')
)

def render_and_upload_image(pyfile: str, image_outfile: str = 'assets/renders/scene.png',
                           cloudinary_folder: str = 'blender_renders'):
    """
    Uploading rendered images (PNG, JPG, etc.)
    
    Args:
        pyfile: Path to the Python script for Blender
        image_outfile: Output path for the rendered image
        cloudinary_folder: Folder name in Cloudinary
    
    Returns:
        dict: Cloudinary upload response
    """
    logger.info(f"Rendering and uploading image to Cloudinary ...")
    blender_bin = '/Applications/Blender.app/Contents/MacOS/Blender'
    
    output_path = pathlib.Path(image_outfile)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    cmd = [blender_bin, '-b', '--python', pyfile]
    logger.info(f"Running Blender: {' '.join(cmd)}")
    
    try:
        subprocess.run(cmd, check=True)
        logger.success(f"Blender rendering completed: {output_path}")
    except subprocess.CalledProcessError as e:
        logger.error(f"Blender rendering failed: {e}")
        raise
    
    try:
        logger.info(f"Uploading {output_path} to Cloudinary...")
        
        upload_result = cloudinary.uploader.upload(
            str(output_path),
            folder=cloudinary_folder,
            public_id=f"{output_path.stem}_{int(time.time())}",
            overwrite=True,
            tags=["blender", "render", "image"],
        )
        
        logger.success(f"Upload successfull {upload_result['public_id']} !!")

        return upload_result
        
    except Exception as e:
        logger.error(f"Cloudinary upload failed: {e}")
        raise

## 6  Vision‑language verifier (stub)

In [61]:
def verify_scene(image_path: str, scene: str) -> float:
    """Return fraction of relations judged correct by GPT-4V (stub)."""

    logger.info(f"Verifying scene ...") 

    system_message = 'You are an expert in evaluating PIL image objects, which contain rooms generated in Blender, and assigning them a score based on their spatial accuracy.'
    prompt = f"Ascribe a decimal number ranging from 10-20 that scores the image in the following link {image_path} based on how well the generated scene satisfies the following JSON layout: {scene}"


    response = OpenAPIClient.chat.completions.create(
            model='o4-mini',
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
    )

    return response.choices[0].message.content # assume perfect

def generate_single_relation_questions(scene_json: str, furniture_catalog: str) -> list[str]:
    """Generate two single-relation spatial questions based on the scene."""
    system_message = 'You are a spatial reasoning expert tasked with generating questions about object relationships in rooms.'

    prompt = f"""Using the furniture catalog below and the layout JSON, create two simple spatial-relation questions (e.g. "Is the chair to the left of the table?").
Furniture catalog: {furniture_catalog}

Scene layout JSON:
{scene_json}
"""

    response = OpenAPIClient.chat.completions.create(
        model='o4-mini',
        messages=[
            {'role':'system', 'content': system_message},
            {'role':'user', 'content': prompt}
        ]
    )

    return response.choices[0].message.content.strip().split('\n')  # Assuming list-like format


def evaluate_questions_on_scene(image_path: str, questions: list[str]) -> list[str]:
    """Ask GPT-4V to answer each spatial question based on the image."""
    system_message = "You're a vision-language model specialized in interpreting room scenes."

    results = []
    for question in questions:
        prompt = f"Given the image at {image_path}, answer the following spatial question:\n\n{question}"

        response = OpenAPIClient.chat.completions.create(
            model='o4-mini',
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ]
        )
        results.append(response.choices[0].message.content.strip())

    return results



In [62]:
import json
from pathlib import Path

def save_benchmark_entry(
    filename: str,
    image_path: str,
    layout: str,
    questions: list[str],
    answers: list[str],
    scene_score: float,
    category: str = "single-relation"
):
    entry = {
        "image": image_path,
        "layout": layout,
        "category": category,
        "scene_score": scene_score,  # 👈 Add this here
        "q_and_a": [{"question": q, "answer": a} for q, a in zip(questions, answers)]
    }

    path = Path(filename)
    if path.exists():
        with open(path, 'r') as f:
            data = json.load(f)
    else:
        data = []

    data.append(entry)

    with open(path, 'w') as f:
        json.dump(data, f, indent=2)

    logger.success(f"Benchmark entry saved to {filename}")


In [63]:
def save_blender_code(code, filename="blender_scene.py"):
    """Save the generated Blender Python code to a file"""
    logger.info(f"Saving Blender Python code to {filename} ...")
    # Create directory if it doesn't exist
    os.makedirs("blender_scripts", exist_ok=True)
    
    # Full path to save the file
    filepath = os.path.join("blender_scripts", filename)
    
    # Write the code to file
    with open(filepath, "w") as f:
        f.write(code)
    
    logger.success(f"Blender Python code saved to: {filepath}")

    return filepath

## 7  End‑to‑end pipeline

In [64]:
def plan_and_render(prompt: str, furniture_catalog: str = "", do_benchmark: bool = False):
    """
    Plan and render with automatic retry on constraint failures.
    After 2 attempts, continues with original positions if constraints fail.
    
    Args:
        prompt: Initial prompt for scene generation
        llm_to_scene: Function to convert prompt to scene JSON
        gen_blender_py: Function to generate Blender Python script
        render_blend: Function to render the Blender file
        verify_scene: Optional function to verify the scene
    """

    logger.info(f"Planning and rendering ...")
    
    attempt = 0
    current_prompt = prompt
    last_error_msg = ""
    last_scene = None
    CONSTRAINT_RETRY_LIMIT = 2  # After this many tries, use original positions
    
    while attempt < MAX_RETRIES:
        attempt += 1
        logger.info(f"Attempt {attempt}/{MAX_RETRIES}")
        
        try:
            logger.info("Generating scene from prompt ...")
            scene = llm_to_scene(current_prompt)
            last_scene = scene  # Save the last generated scene
            logger.success('Scene generated successfully.')
            
            logger.info("Solving constraints ...")
            placements = solve_scene(scene)
            
            if placements is False:
                logger.error(f"Constraint solving failed on attempt {attempt}")
                
                if attempt < CONSTRAINT_RETRY_LIMIT:
                    # Try again with modified prompt
                    last_error_msg = "Previous attempt failed due to unsatisfiable constraints. "
                    current_prompt = f"{prompt}\n\nIMPORTANT: {last_error_msg}Please ensure furniture can fit in the room with proper spacing and avoid conflicting constraints."
                    
                    logger.info("Waiting before retry...")
                    time.sleep(1)  # Brief pause before retry
                    continue
                else:
                    # After 2 tries, just use original positions from the scene
                    logger.warning(f"Constraint solving failed after {CONSTRAINT_RETRY_LIMIT} attempts.")
                
                    
                    # Extract original positions from the scene
                    placements = {}
                    for item in scene.get('furniture', []):
                        if 'id' in item and 'xform' in item and 'pos' in item['xform']:
                            # Use X,Y coordinates from original position
                            placements[item['id']] = (item['xform']['pos'][0], item['xform']['pos'][1])
                        else:
                            # Default position if not specified
                            placements[item['id']] = (1.0, 1.0)
                    
                    logger.info(f"Using original placements: {placements}")
                    
            
            logger.success(f"\n✓ Proceeding with placements: {placements}")
            
            logger.info("\nGenerating Blender script...")
            try:
                py_script = gen_blender_py(scene, placements) 
                logger.success("Blender script generated successfully.")
            except Exception as e:
                logger.error(f"Error generating Blender script: {e}")
                
                if attempt < MAX_RETRIES:
                    current_prompt = f"{prompt}\n\nNote: Blender script generation failed. Please ensure valid scene structure."
                    continue
                else:
                    return None
        except Exception as e:
            logger.error(f"\nUnexpected error: {str(e)}")

            if attempt < MAX_RETRIES:
                current_prompt = f"{prompt}\n\nNote: Previous attempt failed with error: {str(e)}. Please try a different approach."
                time.sleep(1)
                continue
            else:
                logger.error(f"Failed after {MAX_RETRIES} attempts.")
                
                return None
                
    logger.info("Rendering scene...")
    try:
        rendered_scene =  render_and_upload_image(pyfile="assets/blender/script.py", image_outfile="assets/renders/scene.png")
        if not rendered_scene:
            raise ValueError("Render function returned empty result")
            
    except Exception as e:
        logger.error(f"Error during rendering: {e}")

        if attempt < MAX_RETRIES:
            current_prompt = f"{prompt}\n\nNote: Rendering failed with error: {str(e)}. Please try a simpler scene."
        else:
            return None

    logger.info("Verifying scene ...")
    score = verify_scene(rendered_scene['secure_url'], scene)
    if do_benchmark and furniture_catalog:
        logger.info("Generating single-relation benchmark questions...")
        questions = generate_single_relation_questions(json.dumps(scene), furniture_catalog)
        answers = evaluate_questions_on_scene(rendered_scene['secure_url'], questions)
        save_benchmark_entry(
            filename="benchmark_single_relation.json",
            image_path=rendered_scene['secure_url'],
            layout=json.dumps(scene),
            questions=questions,
            answers=answers,
            scene_score=score,
            category="single-relation"
        )

    logger.success(f'Verification score: {score}')            
    logger.success(f"\nSuccess! Scene rendered successfully.")
    return rendered_scene


## 8  Example run

In [65]:
# example_prompt = "Create a bedroom with dimensions 4 x 2 x 3 meters. Include a bed, desk, and lamp. Position furniture logically with proper spacing and accessibility."
# plan_and_render(example_prompt)
furniture_catalog = """
bed: 2m x 1.5m
desk: 1.2m x 0.6m
lamp: 0.3m x 0.3m
"""


example_prompt = (
    "Create a bedroom with dimensions 4 x 2 x 3 meters. "
    "Include a bed, desk, and lamp. Position furniture logically with proper spacing and accessibility."
)

plan_and_render(
    prompt=example_prompt,
    furniture_catalog=furniture_catalog,
    do_benchmark=True
)

2025-06-02 21:21:30.143 | INFO     | __main__:plan_and_render:14 - Planning and rendering ...
2025-06-02 21:21:30.143 | INFO     | __main__:plan_and_render:24 - Attempt 1/5
2025-06-02 21:21:30.143 | INFO     | __main__:plan_and_render:27 - Generating scene from prompt ...
2025-06-02 21:21:56.391 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 21:21:56.395 | SUCCESS  | __main__:plan_and_render:30 - Scene generated successfully.
2025-06-02 21:21:56.395 | INFO     | __main__:plan_and_render:32 - Solving constraints ...
2025-06-02 21:21:56.396 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 21:21:56.463 | SUCCESS  | __main__:plan_and_render:64 - 
✓ Proceeding with placements: {'bed': (0.0, 0.0), 'desk': (1.05, 0.0), 'lamp': (4.0, 0.0)}
2025-06-02 21:21:56.463 | INFO     | __main__:plan_and_render:66 - 
Generating Blender script...
2025-06-02 21:21:56.463 | INFO     | __main__:gen_blender_py:21 - Generating Blender sc

Error: Wall 'west_wall' not found


2025-06-02 21:23:27.541 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 21:23:27.553 | SUCCESS  | __main__:plan_and_render:30 - Scene generated successfully.
2025-06-02 21:23:27.555 | INFO     | __main__:plan_and_render:32 - Solving constraints ...
2025-06-02 21:23:27.555 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 21:23:27.558 | ERROR    | __main__:solve_scene:15 - No constraints found in scene
2025-06-02 21:23:27.558 | ERROR    | __main__:plan_and_render:36 - Constraint solving failed on attempt 4
2025-06-02 21:23:27.558 | WARNING  | __main__:plan_and_render:48 - Constraint solving failed after 2 attempts.
2025-06-02 21:23:27.559 | INFO     | __main__:plan_and_render:61 - Using original placements: {'bed1': (1, 0), 'desk1': (3.2, 0), 'lamp1': (3.5, 0.75)}
2025-06-02 21:23:27.559 | SUCCESS  | __main__:plan_and_render:64 - 
✓ Proceeding with placements: {'bed1': (1, 0), 'desk1': (3.2, 0), 'lamp1': (3.5, 0.75)

Blender 4.4.3 (hash 802179c51ccc built 2025-04-29 14:25:11)
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | Floor
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall_1
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall_2
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall_3
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall_4
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | furn_lamp
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | furn_bed
Fra:1 Mem:10.79M (Peak 10.79M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, Vi

2025-06-02 21:24:20.439 | SUCCESS  | __main__:render_and_upload_image:31 - Blender rendering completed: assets/renders/scene.png
2025-06-02 21:24:20.441 | INFO     | __main__:render_and_upload_image:37 - Uploading assets/renders/scene.png to Cloudinary...



Blender quit


2025-06-02 21:24:21.474 | SUCCESS  | __main__:render_and_upload_image:47 - Upload successfull blender_renders/scene_1748924660 !!
2025-06-02 21:24:21.475 | INFO     | __main__:plan_and_render:104 - Verifying scene ...
2025-06-02 21:24:21.477 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-02 21:24:28.452 | INFO     | __main__:plan_and_render:107 - Generating single-relation benchmark questions...
2025-06-02 21:24:39.780 | SUCCESS  | __main__:save_benchmark_entry:33 - Benchmark entry saved to benchmark_single_relation.json
2025-06-02 21:24:39.782 | SUCCESS  | __main__:plan_and_render:120 - Verification score: 18.3
2025-06-02 21:24:39.782 | SUCCESS  | __main__:plan_and_render:121 - 
Success! Scene rendered successfully.


{'asset_id': '06090f9ab69aabe37cc07f4d1918b7f9',
 'public_id': 'blender_renders/scene_1748924660',
 'version': 1748924660,
 'version_id': 'cdd2b9c6c66d50ee4928e9f425475a67',
 'signature': '997c1fb4d2889a5dde0784dd333d7ddb78ba2242',
 'width': 1920,
 'height': 1080,
 'format': 'png',
 'resource_type': 'image',
 'created_at': '2025-06-03T04:24:20Z',
 'tags': ['blender', 'render', 'image'],
 'bytes': 1407122,
 'type': 'upload',
 'etag': '8f80c430b9494d53cbb9fe057eaf1fdd',
 'placeholder': False,
 'url': 'http://res.cloudinary.com/cld-store/image/upload/v1748924660/blender_renders/scene_1748924660.png',
 'secure_url': 'https://res.cloudinary.com/cld-store/image/upload/v1748924660/blender_renders/scene_1748924660.png',
 'folder': 'blender_renders',
 'original_filename': 'scene',
 'api_key': '986284565458351'}